In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cpu/torch-2.6.0-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cpu/torchvision-0.21.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cpu/torchaudio-2.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 9.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.0 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.

In [1]:
import torch

In [3]:
device="cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device, side_by_side=True)

/Users/buraktekgul/Desktop/anaconda3/lib/python3.11/site-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/bryandlee/animegan2-pytorch/zipball/main" to /Users/buraktekgul/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/face_paint_512_v2.pt" to /Users/buraktekgul/.cache/torch/hub/checkpoints/face_paint_512_v2.pt
100%|██████████| 8.20M/8.20M [00:

In [3]:
import os
import dlib
import collections
from typing import Union,List
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
pip install cmake


  Obtaining dependency information for cmake from https://files.pythonhosted.org/packages/7f/09/516b0d709672bc430eb13278f0316acd34869269447744f5d136daeef689/cmake-3.31.6-py3-none-macosx_10_10_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 5.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install dlib

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for dlib: filename=dlib-19.24.6-cp311-cp311-macosx_13_0_arm64.whl size=2987319 sha256=d1f69214463c3c38492b3d52229f0dfd3dd21e50e5f55d942b22fd4517b630ce
  Stored in directory: /Users/buraktekgul/Library/Caches/pip/wheels/fe/c7/1f/c778b9f7cc6d8d0da4f6697f619f9eb5a49d54d2a2c8267f3c
Successfully built dlib
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def get_dlib_face_detector(predictor_path:str="shape_predictor_68_face_landmarks.dat"):

    if not os.path.isfile(predictor_path):
        model_file="shape_predictor_68_face_landmarks.dat.bz2"
        os.system(f"wget http://dlib.net/files/{model_file}")
        os.system(f"bzip2 -dk {model_file}")

    detector=dlib.get_frontal_face_detector()
    shape_predictor=dlib.shape_predictor(predictor_path)

    def detect_face_landmarks(img:Union[Image.Image,np.ndarray]):

        if isinstance(img,Image.Image):
            img=np.array(img)

        faces=[]
        dets=detector(img)
        for d in dets:
            shape=shape_predictor(img,d)
            faces.append(np.array([[v.x,v.y] for v in shape.parts()]))

        return faces
    
    return detect_face_landmarks #image union nasıl oluyor detector self demeden diğer fonksiyonda nasıl kullandık

def display_facial_landmarks(img:Image,landmarks:List[np.ndarray],fig_size=[15,15]):
    plot_style=dict(
        marker='o',
        markersize=4,
        linestyle='-',
        lw=2
    )

    pred_type=collections.namedtuple('prediction_type',['slice','color'])
    pred_types={
        'face':pred_type(slice(0, 17), (0.682, 0.780, 0.909, 0.5)),
        'eyebrow1':pred_type(slice(17, 22), (1.0, 0.498, 0.055, 0.4)),
        'eyebrow2':pred_type(slice(22, 27), (1.0, 0.498, 0.055, 0.4)),
        'nose':pred_type(slice(27, 31), (0.345, 0.239, 0.443, 0.4)),
        'nostril':pred_type(slice(31, 36), (0.345, 0.239, 0.443, 0.4)),
        'eye1':pred_type(slice(36, 42), (0.596, 0.875, 0.541, 0.3)),
        'eye2':pred_type(slice(42, 48), (0.596, 0.875, 0.541, 0.3)),
        'lips':pred_type(slice(48, 60), (0.596, 0.875, 0.541, 0.3)),
        'teeth':pred_type(slice(60, 68), (0.596, 0.875, 0.541, 0.4))
    }

    fig=plt.figure(figsize=fig_size)
    ax=fig.add_subplot(1,1,1)
    ax.imshow(img)
    ax.axis("off")

    for face in landmarks:
        for pred_type in pred_types.values():
            ax.plot(
                face[pred_type.slice,0],
                face[pred_type.slice,1],
                color=pred_type.color,**plot_style
            )
    
    plt.show()
    